# 모델 테스트

In [1]:
from train import create_dir, load_dataset
from sklearn.metrics import f1_score, jaccard_score
import tensorflow as tf
from tqdm import tqdm
from glob import glob
import pandas as pd
import cv2
import numpy as np
import os

global image_h
global image_w
global num_classes
global classes
global rgb_codes
global cnt
cnt = 0

In [2]:
def grayscale_to_rgb(mask, rgb_codes):
    h, w = mask.shape[0], mask.shape[1]
    mask = mask.astype(np.int32)  # rgb코드에 넣기위해서
    output = []

    for i, pixel in enumerate(mask.flatten()):
        output.append(rgb_codes[pixel])

    output = np.reshape(output, (h, w, 3))
    return output


def save_results(image_x, mask, pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = grayscale_to_rgb(mask, rgb_codes)

    pred = np.expand_dims(pred, axis=-1)
    pred = grayscale_to_rgb(pred, rgb_codes)

    line = np.ones((image_x.shape[0], 10, 3)) * 255

    # cat_images = np.concatenate([image_x, line, mask, line, pred], axis=1)
    print(save_image_path)
    cv2.imwrite(save_image_path, pred)

In [4]:

# 시드
np.random.seed(42)
tf.random.set_seed(42)

# 결과 파일 저장 경로
create_dir("C:\data\lapa\LaPa\\results")

# 파라미터
image_h = 512
image_w = 512
num_classes = 11

# 경로
dataset_path = "C:\data\lapa\LaPa"
model_path = os.path.join("C:\\data\\lapa\\LaPa\\files", "model.h5")

# RGB코드와 클래스
rgb_codes = [
    [0, 0, 0], [0, 153, 255], [102, 255, 153], [0, 204, 153],
    [255, 255, 102], [255, 255, 204], [255, 153, 0], [255, 102, 255],
    [102, 0, 51], [255, 204, 255], [255, 0, 102]
]

classes = [
    "background", "skin", "left eyebrow", "right eyebrow",
    "left eye", "right eye", "nose", "upper lip", "inner mouth",
    "lower lip", "hair"
]

# 데이터 불러오기
(train_x, train_y), (valid_x, valid_y), (test_x,
                                            test_y) = load_dataset(dataset_path)
print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_x)}")
print("")

# 모델 불러오기
model = tf.keras.models.load_model(
    model_path, compile=False)  # complie=True 디폴트, False로 설정 시 모델 구성을 수정할 수 있다. 그러면 model.compile()로 다시 컴파일 해야 한다.

# 프리딕션
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    # 주소에서 파일 이름 자르기
    name = x.split("/")[-1].split(".")[0]

    # 이미지 읽어오기
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (image_w, image_h))
    image_x = image
    image = image/255.0  # (H, W, 3)
    image = np.expand_dims(image, axis=0)  # [1, H, W, 3]
    image = image.astype(np.float32)

    # 마스크 불러오기
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (image_w, image_h))
    mask = mask.astype(np.int32)

    # 예측
    pred = model.predict(image, verbose=0)[0]
    pred = np.argmax(pred, axis=-1)  # [0.1, 0.2, 0.1, 0.6] -> 3
    pred = pred.astype(np.int32)

    # 클래스마다 맞는 컬러 정해주기
    rgb_mask = grayscale_to_rgb(pred, rgb_codes)
    # cv2.imwrite("C:\\data\\lapa\\LaPa\\results\\pred.png", rgb_mask)

    # 마스크 저장하기
    cnt += 1
    save_image_path = f"C:\\data\\lapa\\LaPa\\results\\{cnt}.png"
    # print(save_image_path)
    # save_results(image_x, mask, pred, save_image_path)

    """ Flatten the array """
    mask = mask.flatten()
    pred = pred.flatten()

    labels = [i for i in range(num_classes)]

    """ Calculating the metrics values """
    f1_value = f1_score(mask, pred, labels=labels,
                        average=None, zero_division=0)
    jac_value = jaccard_score(
        mask, pred, labels=labels, average=None, zero_division=0)

    SCORE.append([f1_value, jac_value])

score = np.array(SCORE)
score = np.mean(score, axis=0)

f = open("C:\data\lapa\LaPa\\files\score.csv", "w")
f.write("Class,F1,Jaccard\n")

l = ["Class", "F1", "Jaccard"]
print(f"{l[0]:15s} {l[1]:10s} {l[2]:10s}")
print("-"*35)

for i in range(num_classes):
    class_name = classes[i]
    f1 = score[0, i]
    jac = score[1, i]
    dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
    print(dstr)
    f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f}\n")

print("-"*35)
class_mean = np.mean(score, axis=-1)
class_name = "Mean"

f1 = class_mean[0]
jac = class_mean[1]

dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
print(dstr)
f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f}\n")

f.close()

Train: 18168/18168 - Valid: 2000/2000 - Test: 2000/2000



100%|██████████| 2000/2000 [09:11<00:00,  3.63it/s]

Class           F1         Jaccard   
-----------------------------------
background     : 0.97273 - 0.94866
skin           : 0.94703 - 0.90372
left eyebrow   : 0.57922 - 0.43909
right eyebrow  : 0.60335 - 0.46882
left eye       : 0.56779 - 0.42782
right eye      : 0.55836 - 0.42111
nose           : 0.91294 - 0.85038
upper lip      : 0.69161 - 0.54627
inner mouth    : 0.52613 - 0.42187
lower lip      : 0.73612 - 0.60330
hair           : 0.87514 - 0.80499
-----------------------------------
Mean           : 0.72459 - 0.62146
